In [ ]:
import random
import time
import tracemalloc
import heapq
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# N-queens problem
# N queens on a board, each one on seperate column, solving the problem by implementing following algorithms

In [ ]:
class PriorityQueue:
    def __init__(self):
        self.queue = []
        
    def is_empty(self):
        return len(self.queue) == 0

    def enqueue(self, data):
        heapq.heappush(self.queue, data)
    
    def dequeue(self):
        if not self.is_empty():
            return heapq.heappop(self.queue)
        else:
            print('queue is underflow.')
            return None
        
    def size(self):
        return len(self.queue)

In [ ]:
class NQueens:
    def __init__(self, n, goal_state=None):
        self.n = n
        self.goal_state = goal_state
        self.initial_state = []
        for i in range(n):
            self.initial_state.append(i)
        random.shuffle(self.initial_state)        

In [ ]:
class Vertex:
    def __init__(self, state, path_cost=0, heuristic=0):
        self.state = state
        self.path_cost = path_cost
        self.heuristic = heuristic
        self.f_value = 0
        self.n = len(self.state)

    def __lt__(self, other):
        pass

    # check if 2 pieces are on the same row or diagonal
    def is_collision(self, row1, row2, col1, col2): 
        return row1 == row2 or abs(row1 - row2) == abs(col1 - col2)
    
    # checking goal state
    def is_goal(self): 
        for col1, row1 in enumerate(self.state):
            for col2, row2 in enumerate(self.state):
                if (col1, row1) != (col2, row2) and self.is_collision(row1, row2, col1, col2):
                    return False
        return True
    
    def generate_successors(self, state):
        pass
    
    # calculate path cost
    def g(self):
        return self.path_cost
    
    # calculate heuristic
    def h(self, state):
        attacking_pairs_count = 0
        for col1, row1 in enumerate(state):
            for col2, row2 in enumerate(state[col1+1:], start=col1+1):
                if self.is_collision(row1, row2, col1, col2):
                    attacking_pairs_count += 1
        return attacking_pairs_count
    
    # calculate priority value
    def f(self):
        raise NotImplementedError("Abstract method")

In [ ]:
# vertex for uniform cost search
class UcsVertex(Vertex):
    def __init__(self, state, path_cost=0, heuristic=0, gen_pos=-1):
        super().__init__(state, path_cost, heuristic)
        self.f_value = self.f()
        self.gen_pos = gen_pos
    
    def __lt__(self, other):
        return self.f_value < other.f_value

    def generate_successors(self):
        succs_list = []
        if self.gen_pos == self.n - 1:
            return succs_list
        for col in range(self.n):
            succ_vert = UcsVertex(self.state.copy(), self.path_cost+1, gen_pos=self.gen_pos+1)
            if succ_vert.state[self.gen_pos + 1] == col:
                succ_vert.path_cost -= 1
            succ_vert.state[self.gen_pos + 1] = col
            succs_list.append(succ_vert)
        return succs_list

    def f(self):
        return self.path_cost

In [ ]:
# vertex for A star search
class AstarVertex(Vertex):
    def __init__(self, state, path_cost=0, heuristic=0):
        super().__init__(state, path_cost, heuristic)
        self.f_value = self.f()
        self.heuristic = self.h(self.state)
                 
    def __lt__(self, other):
        return self.f_value < other.f_value

    def generate_successors(self, state):
        succs_list = []
        for col in range(self.n):
            for row in range(self.n):
                if state[col] != row:
                    succ_state = state.copy()
                    succ_state[col] = row
                    succs_list.append(succ_state)
        return succs_list
    
    def f(self):
        return self.path_cost + self.h(self.state)

In [ ]:
class GeneticAlg:
    def __init__(self, initial_state, n):
        self.initial_state = initial_state
        self.n = n

    def heuristic(self, vert: Vertex):
        attacking_pairs_count = 0
        for col1, row1 in enumerate(vert.state):
            for col2, row2 in enumerate(vert.state[col1+1:], start=col1+1):
                if vert.is_collision(row1, row2, col1, col2):
                    attacking_pairs_count += 1
        return attacking_pairs_count

    def max_fitness(self):
        return (self.n - 1) * self.n / 2

    def fitness(self, vert: Vertex):
        return self.max_fitness() - self.heuristic(vert)
    
    def probability(self, vert):
        return self.fitness(vert) / self.max_fitness()

    def find_goal(self, population):
        for vert in population:
            if vert.h(vert.state) == 0:
                return vert
        return None

    def generate_random_state(self):
        new_state = []
        for i in range(self.n):
            new_state.append(i)
        random.shuffle(new_state)
        return new_state

    def generate_population(self, size):
        gen_states = []
        for _ in range(size):
            gen_states.append(Vertex(self.generate_random_state()))
        return gen_states

    def random_selection(self, population: list):
        probability = []
        for i in range(len(population)):
            probability.append(self.probability(population[i]))
        return random.choices(population, weights=probability, k=2)

    def crossover(self, p1, p2):
        cross_point = random.randint(0, self.n - 1)
        c1 = Vertex(p1.state[:cross_point] + p2.state[cross_point:])
        c2 = Vertex(p2.state[:cross_point] + p1.state[cross_point:])
        return c1, c2

    def random_mutation(self, vert: Vertex):
        mutative_position = random.randint(0, self.n - 1)
        mutative_element = random.randint(0, self.n - 1)
        vert.state[mutative_position] = mutative_element


In [ ]:
# Uniform-cost search
def uniform_cost_search(initial_vertex: UcsVertex):
    frontier = PriorityQueue()
    frontier.enqueue(initial_vertex)
    explored = []
    while not frontier.is_empty():
        ucs_vert = frontier.dequeue()
        if ucs_vert.is_goal():
            return ucs_vert.state
        explored.append(ucs_vert.state.copy())
        for s_vert in ucs_vert.generate_successors():
            frontier.enqueue(s_vert)
    print('Solving fail.')
    return None

In [ ]:
# Graph-search A* -- heuristic: number of attacking pairs of queens
def a_star_search(initial_vertex: AstarVertex):
    frontier = PriorityQueue()
    frontier.enqueue((initial_vertex.heuristic, 0, initial_vertex.state))
    explored = []
    while not frontier.is_empty():
        heuristic, path_cost, state = frontier.dequeue()
        as_vert = AstarVertex(state, path_cost, heuristic)
        if heuristic == 0:
            return state
        explored.append(state)
        for s in as_vert.generate_successors(state):
            if s not in explored:
                frontier.enqueue((as_vert.h(s), path_cost + 1, s))
    print('Solving fail.')
    return None

In [ ]:
# Genetic algorithm with the objective function defined from above heuristic
def genetic_algorithm(initial_state: list):
    gen_alg = GeneticAlg(initial_state, len(initial_state))

    pop_size = len(initial_state) 
    population = gen_alg.generate_population(pop_size)
    
    solution = gen_alg.find_goal(population)

    while solution is None:
        new_population = []
        for _ in range(int(pop_size/2)):
            x, y = gen_alg.random_selection(population)

            # generate new generation
            c1, c2 = gen_alg.crossover(x, y)

            if gen_alg.probability(c1) > 0.2:
                gen_alg.random_mutation(c1)
            if gen_alg.probability(c2) > 0.2:
                gen_alg.random_mutation(c2)

            new_population.append(c1)
            new_population.append(c2)

        del population
        population = new_population

        solution = gen_alg.find_goal(population)

    return solution.state

In [ ]:
def print_solution(state, n):
    print("+" + "-" * (2 * n + 1) + "+")

    for row in range(n):
        line = "|"
        for col in range(n):
            if state[col] == row:
                line += " Q"
            else:
                line += " ."
        line += " |"
        print(line)

    print("+" + "-" * (2 * n + 1) + "+")


def solve(alg, n, desc):
    nqp = NQueens(n)
    if desc == n:
        print('init: ' + str(nqp.initial_state))

    # measuring solving time and memory usage
    begin_time = time.time()
    tracemalloc.start()

    # start finding solution
    if alg == 1:
        ucs_v = UcsVertex(nqp.initial_state)
        solution_state = uniform_cost_search(ucs_v)
    elif alg == 2:
        astar_v = AstarVertex(nqp.initial_state)
        solution_state = a_star_search(astar_v)
    else:
        solution_state = genetic_algorithm(nqp.initial_state)


    memory_usage, peak_memory = tracemalloc.get_traced_memory()
    tracemalloc.stop()    

    finish_time = time.time()
    elapsed_time = (finish_time - begin_time) * 1000
    
    memory_usage /= 10**6

    if solution_state is not None and desc == n:
        print('Solution: ', solution_state)
        print("Total time:", elapsed_time, "ms")
        print("Total memory usage:", memory_usage, "MB")
        print_solution(solution_state, n)

    return elapsed_time, memory_usage

In [ ]:
if __name__ == "__main__":
    n = int(input('Number of queens on the board: '))

    print('List of algorithm:')
    print('1. Uniform-cost search')
    print('2. A*')
    print('3. Genetic algorithm')
    sol_algorithm = int(input('Chose an algorithm (number) to solve: '))

    time_list = []
    space_list = []

    for _ in range(5):
        solving_time, space_usage = solve(sol_algorithm, n, n)
        time_list.append(solving_time)
        space_list.append(space_usage)

    print('Average running time: ', sum(time_list)/5, 'ms')
    print('Average memory usage: ', sum(space_list)/5, 'MB')

    # Create a DataFrame, aka a table to store the results
    df = pd.DataFrame(columns=['n', 'time', 'memory'])
    for i in range(4, n+1):
        for _ in range(5):
            solving_time, space_usage = solve(sol_algorithm, i, n)
            df = pd.concat([df, pd.DataFrame({'n': [i], 'time': [solving_time], 'memory': [space_usage]})], ignore_index=True)  

In [ ]:
df

In [ ]:
# Calculate the average performance for each 'N'
avg_time_df = df.groupby('n')['time'].mean().reset_index()

alg_list = {1:'Uniform-cost search', 2:'A*', 3: 'Genetic algorithm'}

plt.figure(figsize=(15, 5))
plt.suptitle(f"Performance of {alg_list[sol_algorithm]}")

## TIME-WISE
plt.subplot(1,2,1) # Estimation of the mean solving time with a 95% confidence interval
sns.lineplot(x='n', y='time', data=df, label='average', color='#edae49')
sns.lineplot(x='n', y='time', data=df.groupby('n')['time'].max().reset_index(), label='max', color='#d1495b')
sns.lineplot(x='n', y='time', data=df.groupby('n')['time'].min().reset_index(), label='min', color='#00798c')
plt.title('Time (ms)')

## MEMORY-WISE
plt.subplot(1,2,2) # Estimation of the mean memory usage with a 95% confidence interval
sns.lineplot(x='n', y='memory', data=df, label='average', color='#edae49')
sns.lineplot(x='n', y='memory', data=df.groupby('n')['memory'].max().reset_index(), label='max', color='#d1495b')
sns.lineplot(x='n', y='memory', data=df.groupby('n')['memory'].min().reset_index(), label='min', color='#00798c')
plt.title('Memory (MB)')

plt.tight_layout()
plt.show()